In [1]:
# cash_flow_analysis.ipynb

import pandas as pd
import plotly.express as px # Using Plotly for interactive charts
import logging
from pathlib import Path

# --- Import functions from your .py file ---
import edgar_analysis_functions as edgar_funcs

# --- Configuration ---
DB_FILE = Path("/Users/dougstrouth/Documents/datasets/edgar/edgar_metadata.duckdb") # <= MAKE SURE THIS PATH IS CORRECT
TARGET_IDENTIFIER = "AAPL" # <= CHANGE TO DESIRED COMPANY TICKER (or CIK if using identifier_type='cik')
IDENTIFIER_TYPE = 'ticker' # 'ticker' or 'cik' or 'name'

# Define the core cash flow tags to analyze
CASH_FLOW_TAGS = [
    'NetCashProvidedByUsedInOperatingActivities',
    'NetCashProvidedByUsedInInvestingActivities',
    'NetCashProvidedByUsedInFinancingActivities',
    'CashAndCashEquivalentsPeriodIncreaseDecrease' # Or a more specific net change tag if needed
]

# --- Setup Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - [%(funcName)s] %(message)s')

# --- 1. Connect to Database ---
db_conn = edgar_funcs.connect_db(DB_FILE)

if db_conn:
    try:
        # --- 2. Get CIK ---
        cik = None
        if IDENTIFIER_TYPE.lower() == 'cik':
            cik = TARGET_IDENTIFIER.zfill(10) # Ensure CIK is 10 digits padded with zeros
        else:
            cik = edgar_funcs.get_company_cik(db_conn, TARGET_IDENTIFIER, identifier_type=IDENTIFIER_TYPE)

        if not cik:
             raise ValueError(f"Could not find CIK for {TARGET_IDENTIFIER}. Exiting.")

        # --- 3. Retrieve Cash Flow Data ---
        cf_data_raw = edgar_funcs.get_cash_flow_data(db_conn, cik, CASH_FLOW_TAGS)

        if cf_data_raw.empty:
            print(f"No cash flow data found for CIK {cik} with the specified tags/forms.")
        else:
            print(f"Raw Data Sample (first 5 rows):\n{cf_data_raw.head().to_markdown(index=False)}\n") # Display sample in Markdown

            # --- 4. Data Cleaning & Preparation ---
            # Handle potential duplicates: Keep the latest filing for the same tag/period_end_date
            cf_data = cf_data_raw.sort_values(by=['period_end_date', 'filed_date'], ascending=[True, True])
            # Keep the last record within each group (latest filed_date for a period_end_date/tag_name)
            cf_data = cf_data.drop_duplicates(subset=['period_end_date', 'tag_name'], keep='last')


            print(f"Cleaned Data Sample (first 5 rows):\n{cf_data.head().to_markdown(index=False)}\n")

            # --- 5. Analysis & Visualization ---

            # Separate Annual (10-K) and Quarterly (10-Q) data
            cf_data_annual = cf_data[cf_data['form'] == '10-K'].copy()
            cf_data_quarterly = cf_data[cf_data['form'] == '10-Q'].copy()

            print(f"Found {len(cf_data_annual)} annual records and {len(cf_data_quarterly)} quarterly records.")

            # Plot Annual Trends
            if not cf_data_annual.empty:
                fig_annual = px.line(
                    cf_data_annual,
                    x='period_end_date',
                    y='value_numeric',
                    color='tag_name',  # Different line color for each cash flow category
                    title=f'Annual Cash Flow Trends for {TARGET_IDENTIFIER} (CIK: {cik}) - 10-K Filings',
                    labels={'period_end_date': 'Period End Date', 'value_numeric': 'Amount (USD)', 'tag_name': 'Cash Flow Category'},
                    markers=True # Add markers to data points
                )
                fig_annual.update_layout(hovermode="x unified")
                fig_annual.show()
            else:
                print("No annual (10-K) data found to plot.")

            # Plot Quarterly Trends
            if not cf_data_quarterly.empty:
                 fig_quarterly = px.line(
                    cf_data_quarterly,
                    x='period_end_date',
                    y='value_numeric',
                    color='tag_name',
                    title=f'Quarterly Cash Flow Trends for {TARGET_IDENTIFIER} (CIK: {cik}) - 10-Q Filings',
                    labels={'period_end_date': 'Period End Date', 'value_numeric': 'Amount (USD)', 'tag_name': 'Cash Flow Category'},
                    markers=True
                )
                 fig_quarterly.update_layout(hovermode="x unified")
                 fig_quarterly.show()
            else:
                 print("No quarterly (10-Q) data found to plot.")

            # --- Further Analysis Ideas ---
            # - Calculate quarterly changes
            # - Pivot data for easier comparison (cf_data.pivot_table(...))
            # - Compare specific periods (e.g., latest Q vs same Q last year)

    except ValueError as ve:
         print(f"Configuration Error: {ve}")
    except Exception as e:
        print(f"An error occurred during analysis: {e}")
        logging.error("Analysis error:", exc_info=True)
    finally:
        if db_conn:
            db_conn.close()
            print("Database connection closed.")
else:
    print("Could not establish database connection. Please check DB_FILE path and database integrity.")

2025-04-19 12:33:40,653 - INFO - [connect_db] Successfully connected to database: /Users/dougstrouth/Documents/datasets/edgar/edgar_metadata.duckdb
2025-04-19 12:33:40,668 - INFO - [get_company_cik] Found CIK 0000320193 for ticker 'AAPL'
2025-04-19 12:33:40,668 - INFO - [get_cash_flow_data] Querying cash flow data for CIK 0000320193, Tags: ['NetCashProvidedByUsedInOperatingActivities', 'NetCashProvidedByUsedInInvestingActivities', 'NetCashProvidedByUsedInFinancingActivities', 'CashAndCashEquivalentsPeriodIncreaseDecrease'], Forms: ['10-K', '10-Q']
2025-04-19 12:33:40,689 - INFO - [get_cash_flow_data] Retrieved 257 cash flow fact records.


Raw Data Sample (first 5 rows):
|        cik | form   | filed_date          | period_end_date     | fp   | tag_name                                     |   value_numeric | unit   |
|-----------:|:-------|:--------------------|:--------------------|:-----|:---------------------------------------------|----------------:|:-------|
| 0000320193 | 10-K   | 2014-10-27 00:00:00 | 2012-09-29 00:00:00 | FY   | CashAndCashEquivalentsPeriodIncreaseDecrease |       9.31e+08  | USD    |
| 0000320193 | 10-K   | 2014-10-27 00:00:00 | 2013-09-28 00:00:00 | FY   | CashAndCashEquivalentsPeriodIncreaseDecrease |       3.513e+09 | USD    |
| 0000320193 | 10-K   | 2015-10-28 00:00:00 | 2013-09-28 00:00:00 | FY   | CashAndCashEquivalentsPeriodIncreaseDecrease |       3.513e+09 | USD    |
| 0000320193 | 10-Q   | 2015-01-28 00:00:00 | 2013-12-28 00:00:00 | Q1   | CashAndCashEquivalentsPeriodIncreaseDecrease |      -1.82e+08  | USD    |
| 0000320193 | 10-Q   | 2015-04-28 00:00:00 | 2014-03-29 00:00:00 | Q2   |

Database connection closed.
